In [ ]:
# Инициируем сессию
from pyspark.sql import SparkSession

spark = (SparkSession.builder
.master("local[*]")
.appName('PySpark_Tutorial')
.getOrCreate()
)

In [71]:
# Читаем JSON файл со странами
countries_json_file = 'world_population.json'
df_countries = spark.read.option('header', True).json(countries_json_file)

In [72]:
# Читаем JSON файл с городами
cities_csv_file = 'worldcities.csv'
df_cities = spark.read.option('header', True).csv(cities_csv_file)

In [75]:
# Приводим код страны к прописным буквам для JOIN'а с таблицей городов
from pyspark.sql.functions import upper
df_countries = df_countries.withColumn("CCA3", upper(df_countries["CCA3"]))

In [78]:
# Конвертируем квадратные мили в квадратные километры в столбцах с площадью и плотность населения
df_countries = df_countries.withColumn("Density (per km²)", 2.58999 * df_countries["Density (per sq mi)"])
df_countries = df_countries.withColumn("Area (km²)", 2.58999 * df_countries["Area (sq mi)"])

In [86]:
# Выполняем JOIN двух таблиц
merged_df = df_cities.join(df_countries, df_cities.iso3 == df_countries.CCA3, how = 'left')

In [85]:
# Оставляем только необходимые столбцы 
merged_df = merged_df.select("city_ascii","country","lat","lng","population").show()

+------------+-------------+--------+--------+----------+
|  city_ascii|      country|     lat|     lng|population|
+------------+-------------+--------+--------+----------+
|       Tokyo|        Japan| 35.6839|139.7744|  39105000|
|     Jakarta|    Indonesia| -6.2146|106.8451|  35362000|
|       Delhi|        India| 28.6667| 77.2167|  31870000|
|      Manila|  Philippines| 14.6000|120.9833|  23971000|
|   Sao Paulo|       Brazil|-23.5504|-46.6339|  22495000|
|       Seoul|  South Korea| 37.5600|126.9900|  22394000|
|      Mumbai|        India| 19.0758| 72.8775|  22186000|
|    Shanghai|        China| 31.1667|121.4667|  22118000|
| Mexico City|       Mexico| 19.4333|-99.1333|  21505000|
|   Guangzhou|        China| 23.1288|113.2590|  21489000|
|       Cairo|        Egypt| 30.0444| 31.2358|  19787000|
|     Beijing|        China| 39.9040|116.4075|  19437000|
|    New York|United States| 40.6943|-73.9249|  18713220|
|     Kolkata|        India| 22.5727| 88.3639|  18698000|
|      Moscow|

In [83]:
merged_df.show(10, vertical=True)

23/02/05 14:41:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
-RECORD 0-----------------------------------------
 city                        | Tokyo              
 city_ascii                  | Tokyo              
 lat                         | 35.6839            
 lng                         | 139.7744           
 country                     | Japan              
 iso2                        | JP                 
 iso3                        | JPN                
 admin_name                  | Tōkyō              
 capital                     | primary            
 population                  | 39105000           
 id                          | 1392685764         
 1970 Population             | 105416839          
 1980 Population             | 117624196          
 1990 Population             | 123686321          
 2000 Population             | 126803861          
 201

In [81]:
df_countries.show(10, vertical=True)

-RECORD 0------------------------------------------
 1970 Population             | 10752971            
 1980 Population             | 12486631            
 1990 Population             | 10694796            
 2000 Population             | 19542982            
 2010 Population             | 28189672            
 2015 Population             | 33753499            
 2020 Population             | 38972230            
 2022 Population             | 41128771            
 Area (sq mi)                | 251827.41086614216  
 CCA3                        | AFG                 
 Capital                     | kabul               
 Continent                   | asia                
 Country/Territory           | afghanistan         
 Density (per sq mi)         | 24.347100184880794  
 Growth Rate                 | 1.0257              
 Rank                        | 36                  
 World Population Percentage | 0.52                
 Density (per km²)           | 63.0587460078394    
 Area (km²) 

In [80]:
df_cities.show(10, vertical=True)

-RECORD 0----------------------
 city       | Tokyo            
 city_ascii | Tokyo            
 lat        | 35.6839          
 lng        | 139.7744         
 country    | Japan            
 iso2       | JP               
 iso3       | JPN              
 admin_name | Tōkyō            
 capital    | primary          
 population | 39105000         
 id         | 1392685764       
-RECORD 1----------------------
 city       | Jakarta          
 city_ascii | Jakarta          
 lat        | -6.2146          
 lng        | 106.8451         
 country    | Indonesia        
 iso2       | ID               
 iso3       | IDN              
 admin_name | Jakarta          
 capital    | primary          
 population | 35362000         
 id         | 1360771077       
-RECORD 2----------------------
 city       | Delhi            
 city_ascii | Delhi            
 lat        | 28.6667          
 lng        | 77.2167          
 country    | India            
 iso2       | IN               
 iso3   